In [1]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_community import embeddings
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate

from langchain.output_parsers import PydanticOutputParser
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import BSHTMLLoader
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler


In [2]:
model_local = ChatOllama(model="mistral",temperature=0)

In [17]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("C:\\Users\\Barani\\Desktop\\local_ollama\\drugs_merged.pdf")
pages = loader.load()

In [18]:
pages

[Document(page_content='HIGHLIGHTS OF PRESCRIBING INFORMATION  \nThese highlights do not include all the information needed to use \nTECENTRIQ safely and effectively. See full prescribing information for \nTECENTRIQ.  \n \nTECENTRIQ® (atezolizumab) injection, for intravenous use  \nInitial U.S. Approval: 2016  \n \n⎯⎯⎯⎯⎯⎯⎯⎯⎯RECENT MAJOR CHANGES ⎯⎯⎯⎯⎯⎯⎯⎯⎯ \nIndication and Usage, Urothelial Carcinoma – \nAccelerated Approval Indication Removed (1.1)  12/2022  \n \nIndications and Usage, Alveolar Soft Par t Sarcoma (1.5)  \nDosage and Administration (2.1, 2.2)  12/2022  \n12/2022  \nDosage and Administration (2.3)  04/2023  \n  \n⎯⎯⎯⎯⎯⎯⎯⎯⎯INDICATIONS AND USAGE ⎯⎯⎯⎯⎯⎯⎯⎯⎯ \nTECENTRIQ is a programmed death -ligand 1 (PD -L1) blocking antibody \nindicated:   \nNon-Small Cell Lung Cancer (NSCLC)  \n•   as adjuvant treatment following resection and platinum -based \nchemotherapy for adult patients with Stage II to IIIA  NSCLC whose \ntumors have  PD-L1 expression on ≥  1% of tumor cells, as det

In [19]:
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=7500, chunk_overlap=200)
doc_splits = text_splitter.split_documents(pages)
len(doc_splits)

149

In [3]:
from langchain_community.vectorstores import FAISS

In [8]:
# vectorstore = Chroma.from_documents(
#             documents=doc_splits,
#             collection_name="rag-chroma",
#             embedding=embeddings.ollama.OllamaEmbeddings(model='nomic-embed-text'),
#         )
# retriever = vectorstore.as_retriever()

In [21]:

embedding = embeddings.ollama.OllamaEmbeddings(model='nomic-embed-text')
db = FAISS.from_documents(doc_splits, embedding)
print(db.index.ntotal)
db.save_local("five_drugs")

149


In [20]:
# 4. After RAG
print("\n########\nAfter RAG\n")
after_rag_template = """provide the answer for the question only from this given context:
{context}
Question: {question}
"""
after_rag_prompt = ChatPromptTemplate.from_template(after_rag_template)
after_rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | after_rag_prompt
    | model_local
    | StrOutputParser()
)



########
After RAG



In [10]:
chunks = []
async for chunk in after_rag_chain.astream("how many employees working in roche?"):
    chunks.append(chunk)
    print(chunk, end="", flush=True)

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


 The text mentions that Roche has "103,613 Employees". Therefore, the answer is 103,613.

In [24]:
async for chunk in after_rag_chain.astream("what is the roche investors website url?"):
    chunks.append(chunk)
    print(chunk, end="", flush=True)

 The Roche Investors website URL is <www.roche.com/investors>.

In [25]:
async for chunk in after_rag_chain.astream("what is Genentech?"):
    chunks.append(chunk)
    print(chunk, end="", flush=True)

 Genentech, Inc. is a biotechnology company founded in 1980 and headquartered in South San Francisco, California. It was acquired by the Swiss pharmaceutical company Roche Group in 2009. Genentech is known for its pioneering work in genetic engineering and the development of various biotechnology products, including therapeutic drugs and diagnostic tools. Some of its notable achievements include the production of insulin using recombinant DNA technology, which revolutionized diabetes care, and the development of the first monoclonal antibody therapy for cancer, Herceptin (trastuzumab). Genentech's focus is on researching, developing, manufacturing, and commercializing medicines to treat various diseases, such as cancer, neurological conditions, liver disease, and respiratory diseases.

# new test

In [1]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_community import embeddings
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate

from langchain.output_parsers import PydanticOutputParser
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import BSHTMLLoader
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.vectorstores import FAISS

In [2]:
model_local = ChatOllama(model="mistral",temperature=0)

In [22]:
embeddings = embeddings.ollama.OllamaEmbeddings(model='nomic-embed-text')
vectorstore = FAISS.load_local("five_drugs", embeddings,allow_dangerous_deserialization=True)

In [23]:
query="recommeded dosage of TECENTRIQ for NSCLC"
new_docs = vectorstore.similarity_search(query)

In [24]:
new_docs

[Document(page_content='2 DOSAGE AND ADMINISTRATION  \n2.1 Patient Selection for Treatment of Non -Small Cell Lung Cancer and Melanoma  \nSelect patients with Stage II to IIIA non -small cell lung cancer for treatment with TECENTRIQ \nas a single agent based on PD -L1 expression on tumor cells [see Clinical Studies (14.1 )]. \nSelect patients with first -line metastatic non -small cell lung cancer for treatment with \nTECENTRIQ as a single agent based on the PD -L1 expression on tumor cells or on tumor -\ninfiltrating immune cells [see Clinical Studies (14.1)].  \nInformation on FDA -approved tests for the determination of PD -L1 expression in metastatic \nnon-small cell lung cancer are available at: http://www.fda.gov/CompanionDiagnostics . \nSelect patients with unresectable or metastatic melanoma  for treatment with TECENTRIQ in \ncombination with cobimetinib and vemurafenib after confirming the presence of a BRAF V600 \nmutation [see Clinical Studies (14.4)]. Information on FDA -ap

In [25]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

In [26]:
# 4. After RAG
print("\n########\nAfter RAG\n")
# after_rag_template = """provide the answer for the question if you find any answer within this context then provide or else then provide your own knowledge:
# {context}
# Question: {question} 
# """
# after_rag_template = """Given the context:
# {context}

# Question: {question}
# only provide the answer for the question from the given context only and do not include your own halucination. and before answering the question make sure you go through all the context i mean if you find any answer at first then dont stop there so search for whole context to give a best answer and before providing the answer make sure the answer make sense of the question asked
# Please provide all instances of the answer from the given context. If there are multiple instances, list them all.
# """
after_rag_template = """
Given the context:
{context}

Question: {question}

Please provide a comprehensive and precise answer based on the given context. Do not infer or add information that isn't present in the context. Ensure to thoroughly examine the entire context before formulating your response, even if an answer is found early on. The answer should directly address the question asked. If multiple instances of the answer exist within the context, please enumerate all of them.
"""

after_rag_prompt = ChatPromptTemplate.from_template(after_rag_template)
after_rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | after_rag_prompt
    | model_local
    | StrOutputParser()
)



########
After RAG



In [27]:
chunks = []
async for chunk in after_rag_chain.astream("recommeded dosage of TECENTRIQ for NSCLC"):
    chunks.append(chunk)
    print(chunk, end="", flush=True)

 TECENTRIQ is recommended for use in adults with non-small cell lung cancer (NSCLC) under the following conditions:

1. As a treatment to help prevent NSCLC from coming back after surgery and platinum-based chemotherapy, if the cancer tests positive for "PD-L1".
2. As a first-line treatment when NSCLC has spread or grown, if it is a non-squamous type and does not have an abnormal "EGFR" or "ALK" gene, along with bevacizumab, paclitaxel, and carboplatin.
3. As a first-line treatment when NSCLC has spread or grown, if it is a non-squamous type and does not have an abnormal "EGFR" or "ALK" gene, along with paclitaxel protein-bound and carboplatin.
4. When NSCLC has spread or grown, if the patient has tried chemotherapy containing platinum and it did not work or is no longer working, and the tumor does not have an abnormal "EGFR" or "ALK" gene.

The recommended dosage for TECENTRIQ in NSCLC is 125 mg administered as an intravenous infusion over 60 minutes every 3 weeks. However, the specif

In [8]:
chunks = []
async for chunk in after_rag_chain.astream("Is there a relationship between pregnancy and breast cancer risk?"):
    chunks.append(chunk)
    print(chunk, end="", flush=True)

 According to the given context, there are several studies and research findings regarding the relationship between pregnancy and breast cancer risk. Here are some key points from the context:

1. Some studies suggest that having a full-term pregnancy or multiple pregnancies may reduce the risk of breast cancer. This is possibly due to the fact that pregnancy alters the hormonal environment in the body, which may have protective effects against breast cancer development.
2. However, other studies indicate that there might be an increased risk of breast cancer in women who have their first full-term pregnancy at a younger age or who have their last full-term pregnancy at an older age. This could be due to the longer exposure to hormones like estrogen during pregnancy and menstruation.
3. The context also mentions that having never been pregnant is a risk factor for endometrial cancer, which is not breast cancer but is related to hormonal imbalances.
4. It's important to note that the re